In [31]:
import os
import json
import numpy as np

from preprocess import frameCoordL

# train_dir = 'D:/signData'
train_dir = '/root/data/signData'
output_dir = f"{train_dir}/nptxt"
json_folder_path = f'{train_dir}/train/label/landmark'


def transShape(array, x, y, flag=True):
    """Reshape and reformat the coordinates."""
    coords = [array[:, i].reshape(x, y) for i in range(3)]  # List of reshaped x, y, z arrays
    if flag:
        return np.stack([c.T[::-1] for c in coords])  # Transform if flag is True
    else:
        return np.stack(coords, axis=0)  # No transformation, stack directly



for person in os.listdir(json_folder_path):
    person_output_path = os.path.join(output_dir, person)
    os.makedirs(person_output_path, exist_ok=True)
    for word in os.listdir(os.path.join(json_folder_path, person)):
        if "F" in word:
            wordCoordL = np.empty((0, 3, 4, 5))  # Initialize arrays for keypoints
            wordCoordR = np.empty((0, 3, 4, 5))
            wordCoordP = np.empty((0, 3, 1, 10))

            for frame in os.listdir(os.path.join(json_folder_path, person, word)):
                file_path = os.path.join(json_folder_path, person, word, frame)

                try:
                    with open(file_path, 'r') as json_file:
                        data = json.load(json_file)
                        lh_points = data['people']['hand_left_keypoints_3d']
                        rh_points = data['people']['hand_right_keypoints_3d']
                        p_points = data['people']['pose_keypoints_3d']

                        # Create normalized arrays for left, right hands, and pose keypoints
                        preFrameCoordP = np.array([[(960 * p_points[j] + 960 - 420) / (1500 - 420), 
                                                    (1080 * p_points[j + 1] + 540) / 1080,
                                                    (p_points[32 + 2] - p_points[i + 2]) / 10]
                                                   for i in range(0, len(p_points), 4)], dtype=np.float32)

                        preFrameCoordL = np.array([[(960 * lh_points[i] + 960 - 420) / (1500 - 420),
                                                    (1080 * lh_points[i + 1] + 540) / 1080,
                                                    (lh_points[2] - lh_points[i + 2]) / 10]
                                                   for i in range(4, len(lh_points), 4)], dtype=np.float32)

                        preFrameCoordR = np.array([[(960 * rh_points[i] + 960 - 420) / (1500 - 420),
                                                    (1080 * rh_points[i + 1] + 540) / 1080,
                                                    (rh_points[2] - rh_points[i + 2]) / 10]
                                                   for i in range(4, len(rh_points), 4)], dtype=np.float32)

                        # Filter specific pose keypoints
                        preFrameCoordP_temp = preFrameCoordP[[i for i in range(21) if (0 <= i <= 7) or (17 <= i <= 18)]]

                        # Transform shapes
                        # frameCoordL = transShape(preFrameCoordL, 5, 4)
                        # frameCoordR = transShape(preFrameCoordR, 5, 4)
                        frameCoordL=frameCoordL.reshape(5,4,3).transpose(1,0,2)#차원을 맨 아래로 내림
                        frameCoordR=frameCoordR.reshape(5,4,3).transpose(1,0,2)
                        frameCoordP = transShape(preFrameCoordP_temp, 1, 10, False)


                        # Append frame data to the word arrays
                                               # Save multiple arrays for the word (Left hand, Right hand, Pose)
                except json.JSONDecodeError as e:
                    print(f"Error reading {file_path}: {e}")
                    break
            wordCoordL = np.append(wordCoordL, [frameCoordL], axis=0)
            wordCoordR = np.append(wordCoordR, [frameCoordR], axis=0)
            wordCoordP = np.append(wordCoordP, [frameCoordP], axis=0)
            word_output_path = os.path.join(person_output_path, f'{word}.npz')
            np.savez(word_output_path, wordCoordL=wordCoordL, wordCoordR=wordCoordR, wordCoordP=wordCoordP)
            print(f"Saved {word_output_path}")
            
            
#todo
#아래 코드를 함수화 하는데 불러올때 포문? 아님 포문돌려서 한번에 다 불러오기?
data = np.load()
wordCoordL = data['wordCoordL']
wordCoordR = data['wordCoordR']
wordCoordP = data['wordCoordP']


FileNotFoundError: [Errno 2] No such file or directory: '/root/data/signData/train/label/landmark'

In [ ]:
import os
import json
import numpy as np
import tensorflow as tf

#train_dir='/home/anaconda/data/signData'
train_dir='D:/signData'
json_folder_path = f'{train_dir}/train/label/landmark'
test=np.empty((0, 3)) 
for i in range(1,21):
    arr=np.array([i,i,i])
    test=np.append(test,[arr],axis=0)



def transShape(array,x,y,flag=True):
    xl_coords=array[:,0]
    yl_coords=array[:,1]
    zl_coords=array[:,2]
    xl_array=xl_coords.reshape(x,y)
    yl_array=yl_coords.reshape(x,y)
    zl_array=zl_coords.reshape(x,y)
    if flag:
        return np.stack([xl_array.T[::-1],yl_array.T[::-1],zl_array.T[::-1]])
    else: return np.stack([xl_array,yl_array,zl_array],axis=0)
        


for person in os.listdir(json_folder_path):
    for word in os.listdir(os.path.join(json_folder_path, person)):
        if "F" in word:
            for frame in os.listdir(os.path.join(json_folder_path, person, word)):
                file_path = os.path.join(json_folder_path,person,word,frame)
                
                preFrameCoordP = np.empty((0, 3))  # Initialize as empty 2D array for 3D coordinates
                preFrameCoordL = np.empty((0, 3))  # For left hand
                preFrameCoordR = np.empty((0, 3))  # For right hand
                frameCoordP = np.empty((0, 3))
                
                
                # Open and read the JSON file
                with open(file_path, 'r') as json_file:
                    try:
                        data = json.load(json_file)  # Parse JSON content
                        lh_points = data['people']['hand_left_keypoints_3d']
                        rh_points = data['people']['hand_right_keypoints_3d']
                        p_points = data['people']['pose_keypoints_3d']
                
                        # Process pose keypoints
                        for j in range(0, len(p_points), 4):
                            xp = 960 *p_points[j] +960   # x-coordinate
                            yp = 1080*p_points[j + 1] + 540    # y-coordinate
                            zp = (p_points[32+2]-p_points[j + 2] ) /10              # z-coordinate
                            arrayP = np.array([xp, yp, zp],dtype=np.float32)
                            
                            # Append to preFrameCoordP
                            preFrameCoordP = np.append(preFrameCoordP, [arrayP], axis=0)
                        
                        # Process left and right hand keypoints
                        for i in range(4, len(lh_points), 4):
                            xl = 960 * lh_points[i] +960   # x-coordinate for left hand
                            yl = 1080 * lh_points[i + 1] +540   # y-coordinate for left hand
                            n_xl=(xl-420)/(1500-420)#normalized x-coordinate for left hand for cut image 1080x1080
                            n_yl=(yl-0)/(1080-0)            
                            zl = (lh_points[2] - lh_points[i + 2]) / 10  # z-coordinate for left hand
                            
                            arrayL = np.array([n_xl, n_yl, zl],dtype=np.float32)
                            preFrameCoordL = np.append(preFrameCoordL, [arrayL], axis=0)
                
                            xr = 960 * rh_points[i] +960  # x-coordinate for right hand
                            yr = 1080 * rh_points[i + 1] +540   # y-coordinate for right hand
                            n_xr=(xr-420)/(1500-420)#normalized x-coordinate for left hand for cut image 1080x1080
                            n_yr=(yr-0)/(1080-0)  
                            zr = (rh_points[2] - rh_points[i + 2]) / 10  # z-coordinate for right hand
                            
                            arrayR = np.array([n_xr, n_yr, zr],dtype=np.float32)
                            preFrameCoordR = np.append(preFrameCoordR, [arrayR], axis=0)
                        
                        for i in range(0,19):
                            frameCoordP=np.append(frameCoordP,[preFrameCoordP[i]], axis=0)
                        

                        frameCoordL=transShape(preFrameCoordL,5,4)
                        frameCoordR=transShape(preFrameCoordR,5,4)
                        frameCoordP=transShape(preFrameCoordP,1,10,False)
                        
                        
                        
                    except json.JSONDecodeError as e:
                        print(f"Error reading {file_path}: {e}")
                        break
            

    

In [30]:
import numpy as np
test_hand=np.empty((0, 3)) 
test_pose=np.empty((0, 3)) 

def transShape(array,x,y,flag=True):
    xl_coords=array[:,0]
    yl_coords=array[:,1]
    zl_coords=array[:,2]
    xl_array=xl_coords.reshape(x,y)
    yl_array=yl_coords.reshape(x,y)
    zl_array=zl_coords.reshape(x,y)
    if flag:
        return np.stack([xl_array.T[::-1],yl_array.T[::-1],zl_array.T[::-1]])
    else: return np.stack([xl_array,yl_array,zl_array],axis=0)
for i in range(0,21):
    arr=np.array([i,i,i])
    if i != 0:
        test_hand=np.append(test_hand,[arr],axis=0)
    if (0<=i<=7) or (17<=i<=18):
        test_pose=np.append(test_pose,[arr],axis=0)
test_hand=test_hand.reshape(5,4,3).transpose(1,0,2)
test_pose=transShape(test_pose,1,10,False)



print(test_hand)
print(test_pose)

[[[ 1.  1.  1.]
  [ 5.  5.  5.]
  [ 9.  9.  9.]
  [13. 13. 13.]
  [17. 17. 17.]]

 [[ 2.  2.  2.]
  [ 6.  6.  6.]
  [10. 10. 10.]
  [14. 14. 14.]
  [18. 18. 18.]]

 [[ 3.  3.  3.]
  [ 7.  7.  7.]
  [11. 11. 11.]
  [15. 15. 15.]
  [19. 19. 19.]]

 [[ 4.  4.  4.]
  [ 8.  8.  8.]
  [12. 12. 12.]
  [16. 16. 16.]
  [20. 20. 20.]]]
[[[ 0.  1.  2.  3.  4.  5.  6.  7. 17. 18.]]

 [[ 0.  1.  2.  3.  4.  5.  6.  7. 17. 18.]]

 [[ 0.  1.  2.  3.  4.  5.  6.  7. 17. 18.]]]


In [ ]:
data = np.load('D:/signData/nptxt/01/NIA_SL_WORD0001_REAL01_F.npz')
wordCoordL = data['wordCoordL']
wordCoordR = data['wordCoordR']
wordCoordP = data['wordCoordP']
print(f"Left Hand Shape: {wordCoordL.shape}")
print(wordCoordL)
print(f"Right Hand Shape: {wordCoordR.shape}")
print(wordCoordR)
print(f"Pose Shape: {wordCoordP.shape}")
print(wordCoordP)